In [13]:
import os, datetime, collections
from typing import Union, Dict
import pandas as pd
from imapclient import IMAPClient
from utils import *
from dotenv import load_dotenv
load_dotenv()

email = os.getenv("user")
password = os.getenv("pass")

In [14]:
list_of_ids = get_ids(email, password, datetime.date(2025, 8, 1)) #year,month,day
print(len(list_of_ids))

219


In [15]:
from concurrent.futures import ThreadPoolExecutor

def fetch_batch(ids):
    with IMAPClient('imap.gmail.com', ssl=True) as server:
        server.login(email, password)
        server.select_folder('INBOX')
        msg_data = server.fetch(ids, ['RFC822', 'ENVELOPE'])
        return msg_data


In [16]:
def get_mail_data(list_of_ids : list[int]) -> list:
    batch_size = 50
    id_chunks = [list_of_ids[i:i+batch_size] for i in range(0, len(list_of_ids), batch_size)]
    with ThreadPoolExecutor(max_workers=7) as executor:
        results = list(executor.map(fetch_batch, id_chunks))
    return results

results = get_mail_data(list_of_ids=list_of_ids)

In [17]:
def make_a_df(results : list[collections.defaultdict[int, Dict[bytes, Union[int, bytes, tuple, imapclient.response_types.Envelope ]]]]) -> pd.core.frame.DataFrame:
    
    out_dict=collections.defaultdict(list) #dict to store the relevant information
    
    for ele in results:         #results ===> list of dicts containing ids as keys, dicts of info as values.
        for x, y in ele.items(): #ele.items() ==> iterable(id of class str, info of class dict)
            out_dict['ids'].append(x)
            out_dict['RFC822'].append(y[b'RFC822'])
            out_dict['ENVELOPE'].append(y[b'ENVELOPE'])
    
    return pd.DataFrame(out_dict) 

In [18]:
out=make_a_df(results)
#out.head(2)

In [19]:
import sys
size_in_bytes=sys.getsizeof(out)

size_in_mb=size_in_bytes / (1024 * 1024)
print(f"Size in MB: {size_in_mb}")

Size in MB: 19.256539344787598
